In [2]:
import SimpleITK as sitk
resample = sitk.ResampleImageFilter()
# resample.SetInterpolator(sitk.sitkNearest)
dir(sitk)

['Abs',
 'AbsImageFilter',
 'AbsoluteValueDifference',
 'AbsoluteValueDifferenceImageFilter',
 'Acos',
 'AcosImageFilter',
 'AdaptiveHistogramEqualization',
 'AdaptiveHistogramEqualizationImageFilter',
 'Add',
 'AddImageFilter',
 'AdditiveGaussianNoise',
 'AdditiveGaussianNoiseImageFilter',
 'AffineTransform',
 'AggregateLabelMap',
 'AggregateLabelMapFilter',
 'And',
 'AndImageFilter',
 'AntiAliasBinary',
 'AntiAliasBinaryImageFilter',
 'ApproximateSignedDistanceMap',
 'ApproximateSignedDistanceMapImageFilter',
 'Asin',
 'AsinImageFilter',
 'Atan',
 'Atan2',
 'Atan2ImageFilter',
 'AtanImageFilter',
 'BSplineDecomposition',
 'BSplineDecompositionImageFilter',
 'BSplineTransform',
 'BSplineTransformInitializer',
 'BSplineTransformInitializerFilter',
 'Bilateral',
 'BilateralImageFilter',
 'BinShrink',
 'BinShrinkImageFilter',
 'BinaryClosingByReconstruction',
 'BinaryClosingByReconstructionImageFilter',
 'BinaryContour',
 'BinaryContourImageFilter',
 'BinaryDilate',
 'BinaryDilateImageFi

In [ ]:
import numpy as np
from glob import glob
from tqdm import tqdm
import h5py
import nrrd
import pandas as pd
import SimpleITK as sitk

output_size =[128, 128, 64]

def resample_image3D(
    image3D,
    newspacing=[0.3,0.3,3],
    newsize=None,
    method='Linear',):
    """做插值"""
    resample = sitk.ResampleImageFilter()
    if method is 'Linear':
        resample.SetInterpolator(sitk.sitkLinear)
    elif method is 'Nearest':
        resample.SetInterpolator(sitk.sitkNearestNeighbor)
    resample.SetOutputDirection(image3D.GetDirection())
    resample.SetOutputOrigin(image3D.GetOrigin())
    resample.SetOutputSpacing(newspacing)

    if not newsize:
        newsize = np.round(np.array(image3D.GetSize())*np.abs(image3D.GetSpacing())/np.array(newspacing)).astype('int').tolist()

    resample.SetSize(newsize)
    # resample.SetDefaultPixelValue(0)

    newimage = resample.Execute(image3D)
#     data1 = sitk.GetArrayFromImage(newimage)
#     print(data1)
    return newimage

# 数组替换元素
def array_replace(array,olds,news):
    #olds:list of old value
    #news:list of new value
    olds = np.array(olds)
    news = np.array(news)
    offset = olds.max()*10
    tmps = olds+offset
    for old,tmp in zip(olds,tmps):
        array[array==old] = tmp
    for tmp,new in zip(tmps,news):
        array[array==tmp] = new
    return array
    
    
def get_stats(glob_str, old_replaced, new_replaced):
    """
    获取数据集的统计信息。
    备注：不要骨头，骨头合并到背景类别中
    """
    listt = glob(glob_str)
    error_samples = []
    stats = pd.DataFrame(columns=['sample_name',
                                  'mean_whole', 
                                  'mean_bg', 
                                  'mean_dura', 
                                  'mean_SC', 
                                  'std_whole',
                                  'std_bg',
                                  'std_dura',
                                  'std_SC',
                                  'old_space0','old_space1','old_space2',
                                  'new_space0','new_space1','new_space2',
                                 ])
    for item in tqdm(listt):
        sample_name = item.split('/')[-2]
        print(sample_name,':')#win系统改为'\\'

        image = sitk.ReadImage(item)
        label = sitk.ReadImage(item.replace(old_replaced, 'Segmentation-label.nrrd'))
        seg = sitk.ReadImage(item.replace(old_replaced, 'Segmentation.seg.nrrd'))
        
        oldspacing = np.abs(image.GetSpacing())
        newspacing = [0.3, 0.3, 3.0]
        print(seg.GetMetaData('Segmentation_ReferenceImageExtentOffset'))
        offset = [int(k) for k in seg.GetMetaData('Segmentation_ReferenceImageExtentOffset').split()]
        print(offset)
        offset = ( np.array(offset)*np.array(oldspacing)/np.array(newspacing) ).astype(int).tolist()
        label_name = [
            seg.GetMetaData('Segment0_Name'),
            seg.GetMetaData('Segment1_Name'),
            seg.GetMetaData('Segment2_Name') 
            ]#人工标注的类别顺序
        
        # resample
        image = resample_image3D(image,newspacing,method='Linear')
        label = resample_image3D(label,newspacing,method='Nearest')#,newsize=image.GetSize()
        seg = resample_image3D(seg,newspacing)
        
        # cut
        sizes = seg.GetSize()#resample后seg的size
        image = sitk.GetArrayFromImage(image).transpose((2,1,0))#tanspose之后才能与sizes匹配
        label = sitk.GetArrayFromImage(label).transpose((2,1,0))#tanspose之后才能与sizes匹配
        seg = sitk.GetArrayFromImage(seg).transpose((3,2,1,0))#tanspose之后才能与sizes匹配
        
        if not image.shape==label.shape:
            import pdb
            pdb.set_trace()
            print('stop')
        assert np.abs(np.array(image.shape)-np.array(label.shape)).sum()<=1, "mismatch shape of image and label"
            
            
        image = image[offset[0]:offset[0]+sizes[0],
                      offset[1]:offset[1]+sizes[1],
                      offset[2]:offset[2]+sizes[2]]
        label = label[offset[0]:offset[0]+sizes[0],
                      offset[1]:offset[1]+sizes[1],
                      offset[2]:offset[2]+sizes[2]].astype(np.uint8)
        
#         import pdb
#         pdb.set_trace()
        print(image.shape)
        # 错误病例：标记的尺寸和image尺寸不同，缺少其中一个类别或者多个类别的标记
        if not np.unique(label).tolist()==[0,1,2,3]:
            error_samples.append(sample_name)
            print("error sample(no dura/SC):",sample_name)
            continue
        
        if not np.abs(np.array(image.shape)-np.array(seg.shape[1::])).sum()<=1:
            error_samples.append(sample_name)
            print("error sample(shape mismatch):",sample_name)
            continue
            
        
        ## 调整顺序，注意：seg是onehot编码
        target_name = ['dura','bone','SC']#目标类别顺序
        idx = [label_name.index(name) for name in target_name]
        idx = np.array(idx)+1
        idx = [0]+idx.tolist()
        label = array_replace(label,olds=[0,1,2,3],news=idx)
        
        # 合并骨头到背景中
        label[label==2] = 0
        label[label==3] = 2
        
        # 统计信息
        
        # std
        std_whole = np.std(image)
        std_bg = np.std(image[label==0])
        std_dura = np.std(image[label==1])
        std_SC = np.std(image[label==2])
        # mean
        mean_whole = np.mean(image)
        mean_bg = np.mean(image[label==0])
        mean_dura = np.mean(image[label==1])
        mean_SC = np.mean(image[label==2])
        
        stats.loc[sample_name]=[sample_name,
                                mean_whole,mean_bg,mean_dura,mean_SC,
                                std_whole,std_bg,std_dura,std_SC,
                                oldspacing[0],oldspacing[1],oldspacing[2],
                                newspacing[0],newspacing[1],newspacing[2],
                               ]   
        
    return error_samples,stats

def covert_h5_unseg(glob_str, old_replaced, new_replaced):
    """
    备注：无标注数据的格式转换
    """
    listt = glob(glob_str)
    for item in tqdm(listt):
        print(item.split('/')[-2],':')
        # 读取原始图
        image, img_header = nrrd.read(item) 
        
#         # 缩小图像
#         image = image[0:-1:2,0:-1:2,:]

#         # 标准化
#         image = (image - np.mean(image)) / np.std(image)
#         image = image.astype(np.float32)
#         print(image.shape)
        
        f = h5py.File(item.replace(old_replaced, new_replaced), 'w')
        f.create_dataset('image', data=image, compression="gzip")
        f.close()  
    print("total number of unseg-samples:", len(listt))

    
if __name__ == '__main__':
    # 有标签数据
    print('seg dataset:')
    glob_str = '../../data/CTM_dataset/Segmented/*/CTM.nrrd'
    error_samples,stats = get_stats(glob_str,'CTM.nrrd','mri_norm2.h5')
    display(stats)
    stats.to_csv('../../data/CTM_dataset/Segmented/stats.csv')
#     # 无标签数据
#     print('unseg dataset:')
#     glob_str = '../../data/CTM_dataset/unSegmented/*/CTM.nrrd'
#     covert_h5_unseg(glob_str,'CTM.nrrd','mri_norm2.h5')
#     glob_str = '../../data/CTM_dataset/unSegmented/*/CT-vol.nrrd'
#     covert_h5_unseg(glob_str,'CT-vol.nrrd','mri_norm2.h5')      
#     error_samples


  0%|          | 0/181 [00:00<?, ?it/s]

seg dataset:
845929 :
0 126 0
[0, 126, 0]
(799, 594, 92)


  1%|          | 1/181 [00:05<16:15,  5.42s/it]

1559027 :
0 32 0
[0, 32, 0]
(492, 461, 73)


  1%|          | 2/181 [00:07<13:06,  4.39s/it]

...

669349 :
0 0 0
[0, 0, 0]
(579, 511, 63)


 62%|██████▏   | 112/181 [05:46<03:10,  2.76s/it]

652658 :
0 132 0
[0, 132, 0]
(558, 340, 70)


 62%|██████▏   | 113/181 [05:48<02:44,  2.42s/it]

B1334915-need revision :
0 0 0
[0, 0, 0]
> <ipython-input-2-1a22f6c59c68>(110)get_stats()
-> print('stop')


In [ ]:
np.round(np.array([1.2,3.72,4.6]))

In [ ]:
error_samples

In [9]:
import SimpleITK as sitk
asd = sitk.ReadImage('../../data/CTM_dataset/Segmented/845929/CTM.nrrd')
# dir(asd)
asd.GetDirection()
# asd.GetOrigin()

(-1.0, 0.0, 0.0, 0.0, -1.0, 0.0, 0.0, 0.0, 1.0)

In [17]:
image=sitk.GetArrayFromImage(asd)#.shape
image.shape
image

array([[[-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        ...,
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048]],

       [[-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        ...,
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048]],

       [[-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        ...,
        [-2048, -2048, -2048, ..., -2048, -2048, -2048],
        [-2048, -2048, -2048, ..., -2048, -20

In [29]:
from skimage import transform
import numpy as np
image_rescale = transform.rescale(image,0.9,preserve_range=True)
np.max(image_rescale)
help(transform.rescale)
# help(transform.resize)

Help on function rescale in module skimage.transform._warps:

rescale(image, scale, order=1, mode='reflect', cval=0, clip=True, preserve_range=False, multichannel=None, anti_aliasing=True, anti_aliasing_sigma=None)
    Scale image by a certain factor.
    
    Performs interpolation to up-scale or down-scale images. Note that anti-
    aliasing should be enabled when down-sizing images to avoid aliasing
    artifacts. For down-sampling N-dimensional images with an integer factor
    also see `skimage.transform.downscale_local_mean`.
    
    Parameters
    ----------
    image : ndarray
        Input image.
    scale : {float, tuple of floats}
        Scale factors. Separate scale factors can be defined as
        `(rows, cols[, ...][, dim])`.
    
    Returns
    -------
    scaled : ndarray
        Scaled version of the input.
    
    Other parameters
    ----------------
    order : int, optional
        The order of the spline interpolation, default is 1. The order has to
        

In [6]:
dir(asd)
# asd.GetOrigin()

['CopyInformation',
 'EraseMetaData',
 'GetDepth',
 'GetDimension',
 'GetDirection',
 'GetHeight',
 'GetITKBase',
 'GetMetaData',
 'GetMetaDataKeys',
 'GetNumberOfComponentsPerPixel',
 'GetNumberOfPixels',
 'GetOrigin',
 'GetPixel',
 'GetPixelAsComplexFloat64',
 'GetPixelID',
 'GetPixelIDTypeAsString',
 'GetPixelIDValue',
 'GetSize',
 'GetSpacing',
 'GetWidth',
 'HasMetaDataKey',
 'IsUnique',
 'MakeUnique',
 'SetDirection',
 'SetMetaData',
 'SetOrigin',
 'SetPixel',
 'SetPixelAsComplexFloat64',
 'SetSpacing',
 'TransformContinuousIndexToPhysicalPoint',
 'TransformIndexToPhysicalPoint',
 'TransformPhysicalPointToContinuousIndex',
 'TransformPhysicalPointToIndex',
 '_Image__ipaste',
 '__GetPixelAsComplexFloat32__',
 '__GetPixelAsDouble__',
 '__GetPixelAsFloat__',
 '__GetPixelAsInt16__',
 '__GetPixelAsInt32__',
 '__GetPixelAsInt64__',
 '__GetPixelAsInt8__',
 '__GetPixelAsUInt16__',
 '__GetPixelAsUInt32__',
 '__GetPixelAsUInt64__',
 '__GetPixelAsUInt8__',
 '__GetPixelAsVectorFloat32__',
 '

In [ ]:
# nrrd.read(item.replace(old_replaced, 'Segmentation-label.nrrd'))
label = sitk.ReadImage('../../data/CTM_dataset/Segmented/845929/Segmentation-label.nrrd')
label.GetSpacing()
# dir(label)
sitk.GetArrayFromImage(asd).shape

In [ ]:
seg = sitk.ReadImage('../../data/CTM_dataset/Segmented/845929/Segmentation.seg.nrrd')
seg.GetSpacing()
dir(seg)
seg.GetSize()
# seg.GetMetaDataKeys()
sitk.GetArrayFromImage(seg).shape

In [ ]:
image3D = asd
newspacing = [0.3, 0.3, 3.0]
resample = sitk.ResampleImageFilter()
resample.SetInterpolator(sitk.sitkLinear)
resample.SetOutputDirection(image3D.GetDirection())
resample.SetOutputOrigin(image3D.GetOrigin())
resample.SetOutputSpacing(newspacing)

newsize = (np.array(image3D.GetSize())*np.abs(image3D.GetSpacing())/np.array(newspacing)).astype(int).tolist()
resample.SetSize(newsize)
# resample.SetDefaultPixelValue(0)

newimage = resample.Execute(image3D)
data1 = sitk.GetArrayFromImage(newimage)
newimage.GetMetaDataKeys()#('Segmentation_ReferenceImageExtentOffset')

In [ ]:
sitk.GetArrayFromImage(asd).transpose((2,1,0)).shape

In [ ]:
seg.GetMetaData('Segmentation_ReferenceImageExtentOffset').split()


In [ ]:
(np.array(seg.GetSize())*np.array(image3D.GetSpacing())/np.array(newspacing)).astype(int)

In [ ]:
seg.GetSize()

In [ ]:
import numpy as np
sitk.GetArrayFromImage(image3D)

In [ ]:
stats['std_SC']/stats['std_whole'].mean()

In [ ]:
newimage.GetSpacing()